In [0]:
%tensorflow_version 2.x 
!pip install -q efficientnet
import os
import gc
import re

import cv2
import math
import numpy as np
import scipy as sp
import pandas as pd

import tensorflow as tf
from IPython.display import SVG
import efficientnet.tfkeras as efn
from keras.utils import plot_model
import tensorflow.keras.layers as L
from keras.utils import model_to_dot
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model,load_model

from tensorflow.keras.applications import DenseNet121

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import files 
files.upload()

In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 3.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=de906f8c2dc1ac52018d052131df515310addcf8e3bd3034f06d656d726f40a2
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 149, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure

In [0]:
!kaggle competitions download -c plant-pathology-2020-fgvc7

100% 776M/779M [00:06<00:00, 147MB/s]
100% 779M/779M [00:06<00:00, 117MB/s]


In [0]:
from zipfile import ZipFile
file_name="plant-pathology-2020-fgvc7.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
EPOCHS = 20
SAMPLE_LEN = 100
IMAGE_PATH = "/content/images"
TEST_PATH = "/content/test.csv"
TRAIN_PATH = "/content/train.csv"
SUB_PATH = "/content/sample_submission.csv"

sub = pd.read_csv(SUB_PATH)
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

In [0]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

BATCH_SIZE = 16 * strategy.num_replicas_in_sync


INFO:tensorflow:Initializing the TPU system: grpc://10.68.202.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.68.202.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
GCS_DS_PATH='gs://kds-44c8a6f93070fa89189427ff10c4c0530ab3f532ac92907ab3c2bb58'
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

test_paths = test_data.image_id.apply(format_path).values
train_paths = train_data.image_id.apply(format_path).values

train_labels = train_data.loc[:, 'healthy':].values
train_paths, valid_paths, train_labels, valid_labels =train_test_split(train_paths, train_labels, test_size=0.15, random_state=2020)



In [0]:
def decode_image(filename, label=None, image_size=(256, 256)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label

In [0]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

In [0]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn


In [0]:
lrfn = build_lrfn()
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

In [0]:
with strategy.scope():
    model = tf.keras.Sequential([DenseNet121(input_shape=(512, 512, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model.summary()

29089792/29084464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 16, 16, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 4100      
Total params: 7,041,604
Trainable params: 6,957,956
Non-trainable params: 83,648
_________________________________________________________________


In [0]:
history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/20
12/12 [==============================] - 67s 6s/step - loss: 1.6371 - categorical_accuracy: 0.1543 - val_loss: 1.8953 - val_categorical_accuracy: 0.0839 - lr: 1.0000e-05

Epoch 00002: LearningRateScheduler reducing learning rate to 8.8e-05.
Epoch 2/20
12/12 [==============================] - 6s 530ms/step - loss: 0.9310 - categorical_accuracy: 0.6341 - val_loss: 2.2419 - val_categorical_accuracy: 0.3321 - lr: 8.8000e-05

Epoch 00003: LearningRateScheduler reducing learning rate to 0.000166.
Epoch 3/20
12/12 [==============================] - 6s 526ms/step - loss: 0.4021 - categorical_accuracy: 0.8665 - val_loss: 3.2537 - val_categorical_accuracy: 0.3431 - lr: 1.6600e-04

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000244.
Epoch 4/20
12/12 [==============================] - 6s 527ms/step - loss: 0.2232 - categorical_accuracy: 0.9277 - val_loss: 3.5603 - val_categorical_accuracy: 0.3869 - l

In [0]:
with strategy.scope():
    model2 = tf.keras.Sequential([efn.EfficientNetB7(input_shape=(256, 256, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(train_labels.shape[1],
                                         activation='softmax')])
        
    model2.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
    model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, 8, 8, 2560)        64097680  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2560)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 10244     
Total params: 64,107,924
Trainable params: 63,797,204
Non-trainable params: 310,720
_________________________________________________________________


In [0]:
history2 = model2.fit(train_dataset,
                    epochs=EPOCHS,
                    callbacks=[lr_schedule],
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_data=valid_dataset)


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/20


ResourceExhaustedError: ignored

In [0]:
probs_densenet = model.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_efnns
sub.to_csv('submission_densenet.csv', index=False)
sub.head()

15/15 [==============================] - 57s 4s/step


,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.000375,0.020581,9.790088e-01,0.000036
1,Test_1,0.000309,0.012880,9.867858e-01,0.000025
2,Test_2,0.000003,0.000212,7.594343e-08,0.999786
3,Test_3,0.996548,0.000050,3.394406e-03,0.000007
4,Test_4,0.000002,0.001849,9.981177e-01,0.000031


In [0]:
probs_efficientnet = model2.predict(test_dataset, verbose=1)
sub.loc[:, 'healthy':] = probs_efficientnet
sub.to_csv('submission_efficient.csv', index=False)
sub.head()

In [0]:
model.save('densenet121.h5')

In [0]:
testing_model=load_model('densenet121.h5')

In [0]:
from keras.preprocessing import image
imagePath='/content/appe-rust.jpg'
imagePath2='/content/apple-rust2.jpg'
imagePath3='/content/apple-scab.jpg'
imagePath4='/content/apple-scab2.jpg'
imagePath5='/content/images/Test_73.jpg'
imagePath6='/content/apple-healthy.jpg'
imagePath7='/content/apple-scab3.jpg'
imagePath8='/content/apple-scab4.jpg'
test_image = image.load_img(imagePath8, target_size = (512, 512)) 
test_image = image.img_to_array(test_image)
test_image1 = np.expand_dims(test_image, axis = 0)

#predict the result
result_classes= model.predict_classes(test_image1)
result=model.predict_proba(test_image1)

In [0]:
"""image=cv2.imread('/content/appe-rust.jpg')
image=cv2.resize(image,(512,512))
image=image.reshape(1,512,512,3)


from google.colab.patches import cv2_imshow"""


"image=cv2.imread('/content/appe-rust.jpg')\nimage=cv2.resize(image,(512,512))\nimage=image.reshape(1,512,512,3)\n\n\nfrom google.colab.patches import cv2_imshow"

In [0]:
result_classes, result

(array([1]),
 array([[5.6240596e-31, 1.0000000e+00, 3.7499852e-09, 0.0000000e+00]],
       dtype=float32))

In [0]:
from keras.models import model_from_json

In [0]:
model_json = model.to_json()
with open("./model.json","w") as json_file:
  json_file.write(model_json)

In [0]:
model.save_weights("model.h5")